### Requirements/Libraries


In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

### Query local Playground dataset

In [5]:
#Loads KG
g = Graph()
g.parse("./data/playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

#Same but with loading a query from a file
print("Query results, Females (query from file):")
query_file = "./data/query.txt"
query = open(query_file, 'r').read()
    
qres = g.query(query)
    
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'
Query results, Females (query from file):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'


### Query local Nobel Prize dataset

In [6]:
#Loads KG
g = Graph()
g.parse("../files/nobelprize_kg.nt", format="nt")
  
print("Loaded '" + str(len(g)) + "' triples.")

#Query with prefixes 
nobelprize_query = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?name_laur WHERE { 
?laur rdf:type nobel:Laureate . 
?laur rdfs:label ?name_laur . 
?laur foaf:gender "female" . }
LIMIT 10
"""

qres = g.query(nobelprize_query)

print("Female laureates (printing 10):")
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("'%s'" % (str(row.name_laur))) #Same name as in sparql query SELECT variable


Loaded '85797' triples.
Female laureates (printing 10):
'Marie Curie, née Sklodowska'
'Maria Goeppert Mayer'
'Irène Joliot-Curie'
'Dorothy Crowfoot Hodgkin'
'Gerty Theresa Cori, née Radnitz'
'Rosalyn Yalow'
'Barbara McClintock'
'Rita Levi-Montalcini'
'Gertrude B. Elion'
'Christiane Nüsslein-Volhard'



### Query remote dataset via SPARQL Enpoint

In [7]:
#Remote service
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
LIMIT 10
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (limit to 10):")

for result in results["results"]["bindings"]:
    #Prints individual results 
    print(result["x"]["value"])
  

Stars co-starring Johnny Depp (limit to 10):

Abigail Breslin
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Aleksandr Kuznetsov
Alfred Molina
Alison Sudol


### Query World Cities Dataset

In [8]:
#Loads KG
g = Graph()
g.parse("./data/worldcities-free-100-task1.ttl", format="ttl")
  
    
print("Loaded '" + str(len(g)) + "' triples.")
    

print("Cities:")
    
qres = g.query(
"""SELECT ?city where {
   ?city rdf:type wco:City .      
}
LIMIT 10
""")
    
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is a City" % (str(row.city)))


Loaded '914' triples.
Cities:
http://www.semanticweb.org/ernesto/in3067-inm713/wco/ahmadabad is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bangalore is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bangkok is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/baoding is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/beijing is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bijie is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/bogota is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/buenos_aires is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/cairo is a City
http://www.semanticweb.org/ernesto/in3067-inm713/wco/cangzhou is a City
